# Extract Transform Load


In [1]:
# !pip install pandas as pd
# !pip install matplotlib
# !pip install nltk
# !pip install fastparquet
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from IPython.display import Markdown, display
%matplotlib inline
def warn(string):
    display(Markdown('<span style="color:red">'+string+'</span>'))
def info(string):
    display(Markdown('<span style="color:blue">'+string+'</span>'))

---
## Load the dataset

In [2]:
wr_df = pd.read_csv('data/winemag-data-130k-v2.csv')
wr_df.head(5)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


---

## Preprocessing

These were the required preprocessing steps define in [reviewed_grapes.data_exp.pandas.v1.ipynb](reviewed_grapes.data_exp.pandas.v1.ipynb):

**We work with lowercase words only**
  - Make all words lowercase
**Decision on duplicates:**
  - Drop entries with duplicated descriptions if the variety is the same
  - Keep entries with duplicated descriptions if the variety is different
**Decision on missing values:**
  - Since we deal with a single missing entry, we just delete it
**Decision on `variety` mention in `description`:**
  - Replace the mention in `description` by the string `'variety'`
**Decision on variety categorization:**
  - We will will only consider the most common wine varieties that make up 90% of the reviews
  - We'll use the varieties as is

In [4]:
# make all words lowercase:
for col in wr_df.columns:
    wr_df[col] = wr_df[col].apply(lambda x: x.lower() if isinstance(x, str) else x)
# drop all rows with duplicated description and variety
wr_filtered = wr_df.drop_duplicates(subset=('description', 'variety'), keep='first')
# delete the row with the missing variety
wr_filtered = wr_filtered.drop(index=wr_filtered[wr_filtered.variety.isnull() == True].index)
# replace the mention of the value in the variety column in the description column
wr_filtered.description = wr_filtered.apply(lambda x: x.description.replace(x.variety, 'variety') if x.variety in x.description else x.description, axis=1)
# now filter for common wine varieties (that make up 90% of the reviews)
nbr_entries = wr_filtered.shape[0]
varieties_df = pd.DataFrame({'count':wr_filtered.groupby('variety', ).variety.count().sort_values(ascending=False)})
varieties_df['fract'] = varieties_df['count']/nbr_entries
fraction = 0
limit_fraction = 0.9
up_to = 1
while fraction <= limit_fraction:
    fraction = varieties_df.iloc[:up_to].fract.sum()
    up_to += 1
common = varieties_df.iloc[:up_to].index.values
print(f'We will use {len(common)} varieties')
wr_filtered = wr_filtered[wr_filtered.variety.isin(common)]

We will use 57 varieties


Finally we want to get rid of the parts we do not need:

In [5]:
# unlink the original dataset:
del(wr_df)
# drop all but the variety and the description columns in the filtered dataset:
for column in wr_filtered.columns:
    if column not in ['description', 'variety']:
        wr_filtered.drop(column, inplace=True, axis=1)
wr_filtered.head(5)

,description,variety
0,"aromas include tropical fruit, broom, brimston...",white blend
1,"this is ripe and fruity, a wine that is smooth...",portuguese red
2,"tart and snappy, the flavors of lime flesh and...",pinot gris
3,"pineapple rind, lemon pith and orange blossom ...",riesling
4,"much like the regular bottling from 2012, this...",pinot noir


## Export to parquet

Now we have our dataset cleaned and ready to be used we export it to a data structure
we can use for the further steps.

In [6]:
# wr_filtered.to_csv('data/reviews_cleaned.csv')
wr_filtered.to_parquet('data/reviews_cleaned', index=True)

---